# Setting Environment

In [8]:
!pip install datasets -q
!pip install transformers -q
!pip install accelerate -q
!pip install python-dotenv -q

In [3]:
# Mounting Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import os
from dotenv import load_dotenv
load_dotenv("creds.env")

project_path = os.getenv("PROJECT_PATH")
os.chdir(project_path)

# Loading Data

In [ ]:
import pandas as pd
from PIL import Image

def load_dataset(image_dir, captions_file):
    # Loading the captions into a DataFrame
    captions_df = pd.read_csv(captions_file, delimiter=',', header=None, names=['image', 'caption'])

    # Dictionary to store the images and captions
    dataset = {}

    # Looping through the DataFrame and load images
    for index, row in captions_df.iterrows():
        image_path = os.path.join(image_dir, row['image'])
        try:
            image = Image.open(image_path)
            # Store the image and caption in the dictionary
            dataset[row['image']] = {'image': image, 'caption': row['caption']}
        except IOError:
            print(f"Error opening {image_path}")

    return dataset

image_dir = 'data/images'
captions_file = 'data/captions.txt'

# Loading the dataset
dataset = load_dataset(image_dir, captions_file)
print("Dataset loaded successfully.")

In [ ]:
# Displaying an example
image_name = list(dataset.keys())[0]
image = dataset[image_name]['image']
caption = dataset[image_name]['caption']

# Display the image and caption
print("Image:", image_name)
print("Caption:", caption)
image.show()